## Productos
- `codigo`: 4 digitos
- `nombre`: 1 a 100 caracteres
- `precio`: 10 a 10000
- `tipo`: 0 a 20 caracteres
- `cantidad`: 0 a 100 
- `valorTotal`: cantidad * precio

In [301]:
class Producto:
    def __init__(self, codigo, nombre, precio, tipo, cantidad): 
        if len(codigo) != 4 or not codigo.isdigit():
            raise ValueError('El codigo debe ser de 4 dígitos')
        self._codigo = codigo
        
        if not (1 <= len(nombre) <= 100):
            raise ValueError('El nombre debe tener entre 1 y 100 caracteres')
        self._nombre = nombre
        
        self.precio = precio  
        self.cantidad = cantidad  
        
        if not (0 <= len(tipo) <= 20):
            raise ValueError('El tipo debe tener hasta 20 caracteres')
        self._tipo = tipo

    @property
    def codigo(self):
        return self._codigo

    @property
    def nombre(self):
        return self._nombre

    @property
    def precio(self):
        return self._precio
    
    @property
    def tipo(self): 
        return self._tipo

    @precio.setter
    def precio(self, nuevoPrecio):
        if 10 <= nuevoPrecio <= 10000:
            self._precio = nuevoPrecio
        else:
            print('El precio debe estar entre 10 y 10000')

    @property
    def cantidad(self):
        return self._cantidad

    @cantidad.setter
    def cantidad(self, nuevaCantidad):
        if 0 <= nuevaCantidad <= 1000:
            self._cantidad = nuevaCantidad
        else:
            print('La cantidad debe estar entre 0 y 1000')

    def valorTotal(self):
        return self.cantidad * self.precio


In [302]:
## NO MODIFIQUE ESTE CODIGO ##

# Prueba de la clase Producto

p1 = Producto('0001', 'Coca Cola', 1500, 'gaseosa', 10)

assert p1.codigo == '0001'
assert p1.nombre == 'Coca Cola'
assert p1.precio == 1500

# Calcula el valor total 
assert p1.valorTotal() == 15000 

# Asegura que los valores de precio y cantidad sean validos
p1.precio = -200
assert p1.precio == 1500    # Rechaza el valor negativo

p1.precio = 2000 
assert p1.precio == 2000

p1.cantidad = -5
assert p1.cantidad == 10    # Rechaza el valor negativo

p1.cantidad = 50
assert p1.cantidad == 50
assert p1.valorTotal() == 100000

print("Prueba pasada exitosamente!")

El precio debe estar entre 10 y 10000
La cantidad debe estar entre 0 y 1000
Prueba pasada exitosamente!


# Ofertas
Debe permitir aplicar oferctas a codigos espeficicos de productos y a tipos de productos
- `descripcion`: 1 a 100 caracteres
- `codigos`: lista de codigos de productos
- `tipos`: lista de tipo de producto
- `esAplicable(producto, cantidad)`: retorna si la oferta es aplicable a ese producto
- `aplicar(producto, cantidad)`: retorna el precio final del producto con la oferta aplicada


In [303]:
class Oferta:
    def __init__(self, descripcion=" ", codigos=[], tipos=[]):
        if not (1 <= len(descripcion) <= 100):
            raise ValueError("La descripción debe tener entre 1 y 100 caracteres")
        self.descripcion = descripcion
        self.codigos = codigos
        self.tipos = tipos

    def esAplicable(self, producto):
        return producto.codigo in self.codigos or producto.tipo in self.tipos
    
    def aplicar(self, producto, cantidad):
        return producto.precio * cantidad


class OfertaDescuento(Oferta):
    def __init__(self, descuento, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.descuento = descuento

    def calcularDescuento(self, producto, cantidad):
        if self.esAplicable(producto):
            return (producto.precio * self.descuento / 100) * cantidad
        return 0


class Oferta2x1:
    def __init__(self, tipos=None, codigos=None):
        self.tipos = tipos or []
        self.codigos = codigos or []
        self.descripcion = "Oferta 2x1"

    def esAplicable(self, producto):
        return producto.codigo in self.codigos or producto.tipo in self.tipos

    def calcularDescuento(self, producto, cantidad):
        # Verificar si la oferta es aplicable al producto
        if self.esAplicable(producto):
            # El descuento equivale a la mitad de los productos adicionales que se entregan gratis
            cantidad_gratis = cantidad // 2
            return cantidad_gratis * producto.precio
        else:
            # Si no es aplicable, el descuento es 0
            return 0

    def calcularCantidadReal(self, producto, cantidad):
        # En una oferta 2x1, por cada 2 productos, el cliente recibe 1 gratis
        cantidad_entregada = cantidad + (cantidad // 2)  # Cada 2 productos, 1 gratis
        return cantidad_entregada




In [304]:
## NO MODIFICAR ESTE CODIGO ##

p1 = Producto('1234', 'Coca Cola', 1000, 'gaseosa', 10)
p2 = Producto('1235', 'Oreo',      2300, 'galleta', 10)

o10d = OfertaDescuento(10, codigos=['1234'])
assert o10d.calcularDescuento(p1, 10) == 1000 
assert o10d.calcularDescuento(p1, 1) == 100

assert o10d.calcularDescuento(p2, 10) == 0

o2x1 = Oferta2x1(tipos=['galleta'])
assert o2x1.calcularDescuento(p1, 10) == 0

assert o2x1.calcularDescuento(p2, 1) == 0
assert o2x1.calcularDescuento(p2, 2) == 2300
assert o2x1.calcularDescuento(p2, 3) == 2300
assert o2x1.calcularDescuento(p2, 4) == 4600
assert o2x1.calcularDescuento(p2, 5) == 4600

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!


# Catalogo
- `leer(archivo) `    : Carga los productos desde el archivo
- `guardar(archivo)`  : Guarda los productos en el archivo
- `agregar(producto)` : Agrega un producto al catalogo
- `buscar(codigo)`    : Busca un producto por codigo o None si no existe
- `registrarOferta(oferta)`  : Registra una oferta
- `buscarOferta(producto, cantidad)`: Busca una oferta por codigo o None si no existe
- `calcularDescuento(producto, cantidad)`: Calcula el descuento de una oferta
- `cantidadProductos`: Retorna la cantidad de productos en el catalogo
- `cantidadUnidades`: Retorna la cantidad de unidades en el catalogo
- `valorTotal`: retorna el valor total del catalogo sin descuentos
- `informe()`: retorna un string con el informe del catalogo 

In [305]:
import csv

class Catalogo:
    def __init__(self):
        self.productos = []
        self.ofertas = []

    def agregar(self, *productos):
        for producto in productos:
            self.productos.append(producto)

    @property
    def cantidadProductos(self):
        return len(self.productos)

    @property
    def cantidadUnidades(self):
        return sum(producto.cantidad for producto in self.productos)

    @property
    def valorTotal(self):
        return sum(producto.precio * producto.cantidad for producto in self.productos)

    def registrarOferta(self, oferta):
        self.ofertas.append(oferta)

    def buscar(self, codigo):
        for producto in self.productos:
            if producto.codigo == codigo:
                return producto
        return None

    def buscarOferta(self, producto, cantidad):
        for oferta in self.ofertas:
            if oferta.esAplicable(producto):
                return oferta
        return None

    def calcularDescuento(self, producto, cantidad):
        oferta = self.buscarOferta(producto, cantidad)
        if oferta:
            return oferta.calcularDescuento(producto, cantidad)
        return 0

    def vender(self, producto, cantidad):
        oferta = self.buscarOferta(producto, cantidad)
        
        if oferta:
            # La cantidad que el cliente recibe
            cantidad_total_entregada = oferta.calcularCantidadReal(producto, cantidad)
            # La cantidad que el cliente realmente paga
            cantidad_a_vender = cantidad
        else:
            cantidad_total_entregada = cantidad
            cantidad_a_vender = cantidad
        
        if producto.cantidad >= cantidad_total_entregada:
            # Descontar solo la cantidad que paga el cliente
            producto.cantidad -= cantidad_a_vender
        else:
            raise ValueError("Cantidad insuficiente para la venta")




    @classmethod
    def leer(cls, archivo):
        nuevo_catalogo = cls()
        with open(archivo, 'r') as file:
            reader = csv.DictReader(file)
            for row in reader:
                producto = Producto(
                    row['codigo'],
                    row['nombre'],
                    float(row['precio']),
                    row['tipo'],
                    int(row['cantidad'])
                )
                nuevo_catalogo.agregar(producto)
        return nuevo_catalogo

    def guardar(self, archivo):
        with open(archivo, 'w', newline='') as file:
            writer = csv.DictWriter(file, fieldnames=['codigo', 'nombre', 'precio', 'tipo', 'cantidad'])
            writer.writeheader()
            for producto in self.productos:
                writer.writerow({
                    'codigo': producto.codigo,
                    'nombre': producto.nombre,
                    'precio': producto.precio,
                    'tipo': producto.tipo,
                    'cantidad': producto.cantidad
                })

    def informe(self):
        informe = f"Cantidad de productos: {self.cantidadProductos}\n"
        informe += f"Cantidad de unidades: {self.cantidadUnidades}\n"
        informe += f"Precio Promedio: $ {self.valorTotal // self.cantidadUnidades:.2f}\n" if self.cantidadUnidades > 0 else "Precio Promedio: $ 0.00\n"
        informe += f"Valor total: $ {self.valorTotal:.2f}\n"

        informe += "Tipos de productos: \n"
        tipos = {}
        for producto in self.productos:
            if producto.tipo not in tipos:
                tipos[producto.tipo] = {'unidades': 0, 'valor': 0}
            tipos[producto.tipo]['unidades'] += producto.cantidad
            tipos[producto.tipo]['valor'] += producto.precio * producto.cantidad

        for tipo, data in tipos.items():
            precio_promedio = data['valor'] / data['unidades'] if data['unidades'] > 0 else 0
            informe += f"  - {tipo}: {data['unidades']}u x $ {precio_promedio:.2f}\n"

        informe += "Ofertas:\n"
        for oferta in self.ofertas:
            informe += f"  - {oferta.descripcion}\n"

        return informe




In [306]:
## NO MODIFIQUE ESTE CODIGO ##

# Prueba del catálogo 

catalogo = Catalogo()
p1 = Producto('0001', 'Coca Cola',  1500, 'gaseosa', 10)
p2 = Producto('0002', 'Pepsi Cola', 1200, 'gaseosa', 20)
p3 = Producto('0003', 'Sonrisa',    1200, 'galleta', 30)
p4 = Producto('0004', 'Oreo',       2300, 'galleta', 40)

## Agregar productos al catalogo 
catalogo.agregar(p1)
catalogo.agregar(p2)
catalogo.agregar(p3)
catalogo.agregar(p4)

assert catalogo.cantidadProductos == 4
assert catalogo.cantidadUnidades == 100

assert catalogo.valorTotal == 167000

## Calcular descuentos segun las ofertas registradas
assert catalogo.calcularDescuento(p1, 5) == 0
assert catalogo.calcularDescuento(p2, 5) == 0

# Ofertas no acumulables 
catalogo.registrarOferta(Oferta2x1(tipos=['galleta']))
catalogo.registrarOferta(OfertaDescuento(10, codigos=['0001', '0003']))

assert catalogo.calcularDescuento(p1, 5) == 750
assert catalogo.calcularDescuento(p2, 5) == 0
assert catalogo.calcularDescuento(p3, 5) == 2400

assert catalogo.valorTotal == 167000.0
catalogo.guardar('catalogo-prueba.csv') ## Guardar datos antes de vender

# Vender afecta la cantidad de unidades y el valor total
catalogo.vender(p3, 3)   

# Verificar que el informe se genere correctamente

informe = catalogo.informe()
assert "Cantidad de productos: " in informe
assert "Cantidad de unidades: " in informe
assert "Precio Promedio: " in informe
assert "Valor total: " in informe
assert "Tipos de productos: " in informe
assert "gaseosa" in informe
assert "galleta" in informe
assert "Ofertas:" in informe 
assert "Oferta 2x1" in informe
assert catalogo.cantidadUnidades == 97
assert catalogo.valorTotal == 163400

# Buscar por código
assert catalogo.buscar('0001') == p1
assert catalogo.buscar('0002') == p2
assert catalogo.buscar('0099') is None 

# Recuperar los datos guardados  
c2 = Catalogo.leer('catalogo-prueba.csv')

assert c2.cantidadProductos == 4
assert c2.cantidadUnidades == 100

# Valor antes de guardar
assert c2.valorTotal == 167000.0

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!


# Cliente
- `nombre`: 1 a 100 caracteres
- `cuit`: 13 digitos (formato XX-XXXXXXXX-X)

In [307]:
import re

class Cliente:
    def __init__(self, nombre, cuit):
        self._nombre = None
        self._cuit = None
        self.nombre = nombre
        self.cuit = cuit

    @property
    def nombre(self):
        return self._nombre

    @nombre.setter
    def nombre(self, valor):
        if 1 <= len(valor) <= 100:
            self._nombre = valor
        else:
            print("Error: El nombre debe tener entre 1 y 100 caracteres.")

    @property
    def cuit(self):
        return self._cuit

    @cuit.setter
    def cuit(self, valor):
        if re.match(r"^\d{2}-\d{8}-\d$", valor):
            self._cuit = valor
        else:
            print("Error: El CUIT debe tener el formato XX-XXXXXXXX-X.")

         
        


In [308]:
## NO MODIFICAR ESTE CODIGO ##

# Prueba de la clase Cliente #

c1 = Cliente('Juan Perez', '20-12345678-1')

assert c1.nombre == 'Juan Perez'
assert c1.cuit   == '20-12345678-1'

c1.nombre = ''
assert c1.nombre == 'Juan Perez' # Rechaza el valor vacio

c1.nombre = 'Juana Perez'        # Acepta el nuevo valor
assert c1.nombre == 'Juana Perez'

c1.cuit = '1234567890123'
assert c1.cuit == '20-12345678-1' # Rechaza el valor incorrecto

c1.cuit = 'CC-12345678-1'
assert c1.cuit == '20-12345678-1' # Rechaza el valor incorrecto

print("Prueba pasada exitosamente!")

Error: El nombre debe tener entre 1 y 100 caracteres.
Error: El CUIT debe tener el formato XX-XXXXXXXX-X.
Error: El CUIT debe tener el formato XX-XXXXXXXX-X.
Prueba pasada exitosamente!


In [309]:
from datetime import datetime

class Factura:
    contador_facturas = 100 

    @classmethod
    def ultimaFactura(cls, numero):
        cls.contador_facturas = numero

    @classmethod
    def nuevoNumero(cls):
        cls.contador_facturas += 1
        return cls.contador_facturas

    def __init__(self, catalogo, cliente):
        self.catalogo = catalogo
        self.cliente = cliente
        self.numero = Factura.nuevoNumero()
        self.fecha = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        self.productos_vendidos = []

    @property
    def cantidadProductos(self):
        return len(set(producto for producto, _ in self.productos_vendidos))

    @property
    def cantidadUnidades(self):
        return sum(cantidad for _, cantidad in self.productos_vendidos)

    @property
    def subtotal(self):
        return sum(producto.precio * cantidad for producto, cantidad in self.productos_vendidos)

    @property
    def descuentos(self):
        total_descuentos = 0
        for producto, cantidad in self.productos_vendidos:
            total_descuentos += self.catalogo.calcularDescuento(producto, cantidad)
        return total_descuentos

    @property
    def total(self):
        return self.subtotal - self.descuentos

    def agregar(self, producto, cantidad):
        if cantidad > 0 and producto.cantidad >= cantidad:
            for i in range(len(self.productos_vendidos)):
                if self.productos_vendidos[i][0] == producto:
                    nuevo_cantidad = self.productos_vendidos[i][1] + cantidad
                    if nuevo_cantidad <= producto.cantidad:  
                        producto.cantidad -= cantidad 
                        self.productos_vendidos[i] = (producto, nuevo_cantidad)
                        return
            
            self.productos_vendidos.append((producto, cantidad))
            producto.cantidad -= cantidad 

    def imprimir(self):
        factura_texto = f"Factura: {self.numero}\n"
        factura_texto += f"Fecha  : {self.fecha}\n"
        factura_texto += f"Cliente: {self.cliente.nombre} ({self.cliente.cuit})\n\n"

        productos_agrupados = {}

        for producto, cantidad in self.productos_vendidos:
            if producto in productos_agrupados:
                productos_agrupados[producto] += cantidad
            else:
                productos_agrupados[producto] = cantidad

        for producto, cantidad in productos_agrupados.items():
            subtotal_producto = producto.precio * cantidad
            descuento_aplicado = self.catalogo.calcularDescuento(producto, cantidad)

            factura_texto += f"{cantidad} {producto.nombre:<30} x ${producto.precio:.2f} = ${subtotal_producto:.2f}\n"
        
        if descuento_aplicado > 0:
            if producto.codigo in ['0001', '0003']:
                factura_texto += f"Descuento 10% - ${descuento_aplicado:.2f}\n"
            if producto.tipo == 'galleta':
                factura_texto += f"Oferta 2x1 - ${descuento_aplicado:.2f}\n"

        factura_texto += f"\n{' ' * 41}Subtotal:   ${self.subtotal:.2f}\n"
        factura_texto += f"{' ' * 41}Descuentos: ${self.descuentos:.2f}\n"
        factura_texto += f"{' ' * 41}-----------------------\n"
        factura_texto += f"{' ' * 41}TOTAL:      ${self.total:.2f}\n"

        return factura_texto

In [310]:
## NO MODIFICAR ESTE CODIGO ##

# Prueba de la clase Factura #

# Creo un catálogo con productos
catalogo = Catalogo()
p1 = Producto('0001', 'Coca Cola',  1500, 'gaseosa', 10)
p2 = Producto('0002', 'Pepsi Cola', 1200, 'gaseosa', 20)
p3 = Producto('0003', 'Sonrisa',    1200, 'galleta', 30)
p4 = Producto('0004', 'Oreo',       2300, 'galleta', 40)
catalogo.agregar(p1,p2,p3,p4)

# Registro ofertas
catalogo.registrarOferta(Oferta2x1(tipos=['galleta']))
catalogo.registrarOferta(OfertaDescuento(10, codigos=['0001', '0003']))

# Creo un cliente
cliente = Cliente('Juan Perez', '20-12345678-9')

# Creo una factura
Factura.ultimaFactura(100)
assert Factura.nuevoNumero() == 101
assert Factura.nuevoNumero() == 102

f1 = Factura(catalogo, cliente)
f1.agregar(p1, 5)
f1.agregar(p3, 3)

assert f1.numero == 103
assert f1.cantidadProductos == 2
assert f1.cantidadUnidades  == 8

# Agrega unidades de un producto ya agregado
f1.agregar(p1, 5)
assert f1.cantidadProductos == 2
assert f1.cantidadUnidades == 13

assert f1.subtotal   == 18600
assert f1.descuentos == 2700.0
assert f1.total == 15900.0

impresion = f1.imprimir()

assert "Juan Perez" in impresion
assert "10 Coca Cola" in impresion
assert "Sonrisa" in impresion
assert "Descuento 10%" in impresion
assert "Oferta 2x1" in impresion
assert "TOTAL:" in impresion
assert "15900.00" in impresion

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!


In [311]:
## NO MODIFICAR ESTE CODIGO ##

# Prueba de integración #

# Cargamos los datos
catalogo = Catalogo.leer('catalogo.csv')
juan  = Cliente('Juan Perez', '20-12345678-9')
maria = Cliente('Maria Lopez', '27-87654321-3')

o2x1 = Oferta2x1(tipos=['galleta'], codigos=['0002', '0003','0010'])
od20 = OfertaDescuento(20, codigos=['0001', '0002'], tipos=['gaseosa', 'arroz'])
od10 = OfertaDescuento(10, tipos=['fideo'])

catalogo.registrarOferta(o2x1)
catalogo.registrarOferta(od20)
catalogo.registrarOferta(od10)

# Controlo que la carga este correcta
assert catalogo.cantidadProductos == 30
assert catalogo.cantidadUnidades == 1000
assert catalogo.valorTotal == 2000000


Factura.ultimaFactura(10000)

# Crear una factura
f1 = Factura(catalogo, juan)
f1.agregar(catalogo.buscar('0001'), 5)
f1.agregar(catalogo.buscar('0002'), 3)
f1.agregar(catalogo.buscar('0003'), 2)

assert f1.numero == 10001
assert f1.cantidadProductos == 3
assert f1.cantidadUnidades == 10
assert f1.subtotal == 13450.0
assert f1.descuentos == 3890.0
assert f1.total == 9560.0

assert catalogo.cantidadUnidades == 990

# Crear otra factura
f2 = Factura(catalogo, maria)
f2.agregar(catalogo.buscar('0010'), 5)
f2.agregar(catalogo.buscar('0010'), 3)
f2.agregar(catalogo.buscar('0020'), 2)
f2.agregar(catalogo.buscar('0030'), 2)

assert f2.numero == 10002
assert f2.cantidadProductos == 3
assert f2.cantidadUnidades == 12
assert f2.subtotal == 23900.00
assert f2.descuentos == 8860.00
assert f2.total == 15040.00

assert catalogo.cantidadUnidades == 978

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!
